In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
pd.set_option('display.max_columns', None)

In [5]:
# Lectura de datos
df = pd.read_parquet("../Preprocessing/flightsFilteredCleaned.parquet")

In [ ]:
pd.read_parquet(str(os.getcwd())+"\\Preprocessing\\flightsFilteredCleaned.parquet")

Queremos predecir los retrasos correspondientes con los 5 aeropuertos que experimentan mayor número de retrasos. En primer lugar debemos seleccionar dichos aeropuertos. Para ello calcularemos el número total de vuelos, el número total de vuelos retrasados y el % de vuelos retrasados para cada uno de los aeropuertos. Una vez hecho esto nos quedaremos con aquellos 10 que más volumen de tráfico generen. De esos 10 filtraremos nuevamente hasta quedarnos con los 5 que muestren un % de retrasos más elevado

In [4]:
# Agrupos los datos de vuelos por aeropuesrtos
airports = pd.DataFrame(df.groupby(df['ORIGIN_AIRPORT'])["FLIGHT_NUMBER"].count())
airports = airports.rename(columns={"FLIGHT_NUMBER":"TOTAL_FLIGHTS"})
airports["DELAYED_FLIGHTS"] =  df[df["ARRIVAL_DELAY"]>0].groupby(df['ORIGIN_AIRPORT'])["FLIGHT_NUMBER"].count()
airports["DELAY_PERCENTAGE"] = np.round(airports['DELAYED_FLIGHTS']/airports['TOTAL_FLIGHTS']*100,2)

# Ordenamos los aeropuertos de tal manera que los 5 con mayor % de retrasos aparezcan los primeros
airports = airports.sort_values('TOTAL_FLIGHTS',ascending=False)
airports = airports.reset_index(level=0, drop=False)

# Seleccionamos el top 10 trafico aereo
top_10 = airports.loc[0:9]
top_10 = list(top_10['ORIGIN_AIRPORT'])

# Filtramos nuestros datos para quedarnos únicamente con los correspondientes a dichos aeropuertos
top_10_df = df[df.ORIGIN_AIRPORT.isin(top_10)]

In [5]:
# De estos diez, cogemos los 5 con mayor numero de retrasos
# Repetimos el proceso
del(airports)

# Agrupos los datos de vuelos por aeropuesrtos
df = top_10_df
airports = pd.DataFrame(df.groupby(df['ORIGIN_AIRPORT'])["FLIGHT_NUMBER"].count())
airports = airports.rename(columns={"FLIGHT_NUMBER":"TOTAL_FLIGHTS"})
airports["DELAYED_FLIGHTS"] =  df[df["ARRIVAL_DELAY"]>0].groupby(df['ORIGIN_AIRPORT'])["FLIGHT_NUMBER"].count()
airports["DELAY_PERCENTAGE"] = np.round(airports['DELAYED_FLIGHTS']/airports['TOTAL_FLIGHTS']*100,2)

# Ordenamos los aeropuertos de tal manera que los 5 con mayor % de retrasos aparezcan los primeros
airports = airports.sort_values('DELAYED_FLIGHTS',ascending=False)
airports = airports.reset_index(level=0, drop=False)


# Seleccionamos el top 10 trafico aereo
top_5 = airports.loc[0:4]
top_5 = list(top_5['ORIGIN_AIRPORT'])

# Filtramos nuestros datos para quedarnos únicamente con los correspondientes a dichos aeropuertos
top_5_delay = df[df.ORIGIN_AIRPORT.isin(top_5)]
len(top_5_delay)

1353729

In [6]:
list(top_5_delay['ORIGIN_AIRPORT'].unique())

['LAX', 'DEN', 'ORD', 'DFW', 'ATL']

In [7]:
# Guardamos
top_5_delay.to_parquet("top_5_delay.parquet", index=False)